In [1]:
import pandas as pd, numpy as np, json

In [43]:
census = pd.read_csv("http://www.census.gov/popest/data/counties/asrh/2015/files/CC-EST2015-ALLDATA-09.csv")
census.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,9,1,Connecticut,Fairfield County,1,0,916829,445601,471228,...,70171,67235,7428,7882,2393,2100,915,830,527,403
1,50,9,1,Connecticut,Fairfield County,1,1,56899,29226,27673,...,6110,5833,994,931,234,192,124,93,45,31
2,50,9,1,Connecticut,Fairfield County,1,2,62907,32165,30742,...,5742,5281,839,817,199,188,101,77,25,32
3,50,9,1,Connecticut,Fairfield County,1,3,66606,34212,32394,...,5539,5384,739,804,171,169,88,94,47,43
4,50,9,1,Connecticut,Fairfield County,1,4,62192,31816,30376,...,5707,5151,759,738,198,176,69,73,35,31


In [13]:
names = census.dtypes.to_frame().reset_index()
names.columns = "name","type"

In [41]:
county_names = census["CTYNAME"].unique()
county_names

array(['Fairfield County', 'Hartford County', 'Litchfield County',
       'Middlesex County', 'New Haven County', 'New London County',
       'Tolland County', 'Windham County'], dtype=object)

In [44]:
age_groups = census["AGEGRP"].unique()
age_groups

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [45]:
years = census["YEAR"].unique()
years

array([1, 2, 3, 4, 5, 6, 7, 8])

In [223]:
# return true if male pop plus female pop equals total pop
def checksum(df):
    return df.apply(lambda x: x["TOT_POP"] - x["TOT_MALE"] - x["TOT_FEMALE"], axis=1).sum() == 0

def county_summary(county, year):
    county_df = census[(census["CTYNAME"] == county) &
                      (census["YEAR"] == year)]
    total_pop = census[(census["CTYNAME"] == county) &
                      (census["YEAR"] == year) &
                      (census["AGEGRP"] == 0)]["TOT_POP"]
    if checksum(county_df):
        county_df["PCT_FEMALE"] = county_df.apply(lambda x: int(x["TOT_FEMALE"]) * 100 / total_pop, axis=1).fillna("null")
        county_df["PCT_MALE"] = county_df.apply(lambda x: 0 - int(x["TOT_MALE"]) * 100 / total_pop, axis=1).fillna("null")

        return county_df[["CTYNAME","YEAR","AGEGRP","TOT_MALE","TOT_FEMALE","TOT_POP","PCT_FEMALE","PCT_MALE"]]
    else:
        return False

def county_summary_current(county):
    return county_summary(county, years[len(years)-1])



In [255]:
## Generate the data array for a given gender and dataframe
def data_array(df, sex):
    ret = []
    #if sex == "MALE":
    #    df.apply(lambda x: ret.append(0 - int(x["TOT_" + sex])), axis=1)
    #else:
    df.apply(lambda x: ret.append(x["PCT_" + sex]), axis=1)
    return ret

# Sorry for non-DRY -- quick add.
def data_array_total(df, sex):
    ret = []
    #if sex == "MALE":
    #    df.apply(lambda x: ret.append(0 - int(x["TOT_" + sex])), axis=1)
    #else:
    df.apply(lambda x: ret.append(x["TOT_" + sex]), axis=1)
    return ret

             
data_array(ffld, "MALE")
#data_array(ffld, "FEMALE")

[-48.722170595947695,
 -2.8628146316714362,
 -3.2838881370556288,
 -3.548008391935894,
 -3.6580233383576655,
 -3.1928594709367513,
 -2.837499591267577,
 -2.9610158925714067,
 -3.011962411384174,
 -3.19612933032225,
 -3.663719222448534,
 -3.8428231333058385,
 -3.587774101236956,
 -2.8399256159729465,
 -2.1055784855909954,
 -1.525969539677634,
 -1.0247317396812203,
 -0.7628265508362929,
 -0.8166210116944939]

In [256]:
## For every county, for every year, for each gender, generate a data array
def summarize():
    ret = {}
    for county in county_names:
        ret[county] = {}
        for year in years:
            ret[county][year] = {
                "MALE" : data_array(county_summary(county,year),"MALE"),
                "FEMALE" : data_array(county_summary(county,year),"FEMALE")
            }
    return ret

# Sorry for non-DRY -- quick add.

def summarize_totals():
    ret = {}
    for county in county_names:
        ret[county] = {}
        for year in years:
            ret[county][year] = {
                "MALE" : data_array_total(county_summary(county,year),"MALE"),
                "FEMALE" : data_array_total(county_summary(county,year),"FEMALE")
            }
    return ret

all_year_data_totals = summarize_totals()
all_year_data = summarize()

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [257]:
data_file = open ("viz/js/all_year_data.json","w")
data_file.write(json.dumps(all_year_data,indent=2))
data_file.close()

data_file = open ("viz/js/all_year_data_totals.json","w")
data_file.write(json.dumps(all_year_data_totals,indent=2))
data_file.close()

In [249]:
census_min = census[["CTYNAME","YEAR","AGEGRP","TOT_POP",
                     "TOT_FEMALE","TOT_MALE"]]
census_min

,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_FEMALE,TOT_MALE
0,Fairfield County,1,0,916829,471228,445601
1,Fairfield County,1,1,56899,27673,29226
2,Fairfield County,1,2,62907,30742,32165
3,Fairfield County,1,3,66606,32394,34212
4,Fairfield County,1,4,62192,30376,31816
5,Fairfield County,1,5,50516,24632,25884
6,Fairfield County,1,6,52251,25806,26445
7,Fairfield County,1,7,53812,27305,26507
8,Fairfield County,1,8,60121,30937,29184
9,Fairfield County,1,9,71125,36876,34249


In [250]:

census_min["id"] = census_min.apply(lambda x: x["CTYNAME"] + ":" + str(x["AGEGRP"]), axis=1)
#census_min["id"] = census_min["CTYNAME"] + census_min["AGEGRP"].str.lower()
men_change = census_min.pivot(index="id",
                              columns="YEAR",
                              values="TOT_MALE")

women_change = census_min.pivot(index="id",
                              columns="YEAR",
                              values="TOT_FEMALE")

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [214]:
men_change

YEAR,1,2,3,4,5,6,7,8
id,,,,,,,,
Fairfield County:0,445601,445611,447123,452077,455256,458593,460762,461912
Fairfield County:1,29226,29228,29230,28853,28185,27687,27421,27141
Fairfield County:10,37237,37237,37224,37042,36635,35907,35140,34734
Fairfield County:11,35084,35083,35253,35965,36319,36480,36598,36432
Fairfield County:12,28621,28623,28926,30147,31514,32639,33453,34014
Fairfield County:13,22969,22970,23140,24016,23905,24632,25700,26924
Fairfield County:14,16940,16939,17088,17620,18930,19333,19591,19962
Fairfield County:15,11466,11466,11543,11981,12522,13468,13975,14467
Fairfield County:16,9505,9505,9452,9363,9341,9413,9574,9715


In [160]:
women_change

YEAR,1,2,3,4,5,6,7,8
id,,,,,,,,
Fairfield County:0,471228,471239,472621,476936,479982,483200,485054,486141
Fairfield County:1,27673,27674,27671,27330,26934,26577,26342,26142
Fairfield County:10,40125,40126,40161,39910,39416,38959,37695,37016
Fairfield County:11,36840,36839,37031,37974,38575,38922,39391,39353
Fairfield County:12,29993,29994,30330,31666,32991,34091,35037,35897
Fairfield County:13,25456,25456,25624,26455,26263,26817,27705,28706
Fairfield County:14,19365,19365,19499,20023,21476,21955,22504,23070
Fairfield County:15,14570,14572,14623,14966,15568,16457,16822,17228
Fairfield County:16,12642,12642,12576,12399,12334,12408,12640,12797


In [179]:
def diffs(df):
    return df.apply(lambda x: (x[8] - x[7])* 100 / x[7] , axis=1)

men_change["diff_15"] = diffs(men_change)
women_change["diff_15"] = diffs(women_change)

In [180]:
men_change

YEAR,1,2,3,4,5,6,7,8,diff_15
id,,,,,,,,,
Fairfield County:0,445601,445611,447123,452077,455256,458593,460762,461912,0
Fairfield County:1,29226,29228,29230,28853,28185,27687,27421,27141,-2
Fairfield County:10,37237,37237,37224,37042,36635,35907,35140,34734,-2
Fairfield County:11,35084,35083,35253,35965,36319,36480,36598,36432,-1
Fairfield County:12,28621,28623,28926,30147,31514,32639,33453,34014,1
Fairfield County:13,22969,22970,23140,24016,23905,24632,25700,26924,4
Fairfield County:14,16940,16939,17088,17620,18930,19333,19591,19962,1
Fairfield County:15,11466,11466,11543,11981,12522,13468,13975,14467,3
Fairfield County:16,9505,9505,9452,9363,9341,9413,9574,9715,1


## Biggest changes

In [195]:
men_change[men_change["diff_15"] >= 5].sort_values(by="diff_15").reset_index()

YEAR,id,1,2,3,4,5,6,7,8,diff_15
0,Windham County:17,854,854,862,882,893,886,849,897,5
1,Tolland County:18,847,847,854,889,931,957,958,1022,6
2,Tolland County:16,1345,1345,1353,1402,1433,1475,1558,1669,7


In [190]:
men_change[men_change["diff_15"] <= -5].sort_values(by="diff_15").reset_index()

YEAR,id,1,2,3,4,5,6,7,8,diff_15
0,Litchfield County:9,7165,7165,7094,6807,6399,6066,5635,5150,-9
1,Middlesex County:9,6248,6248,6204,6045,5751,5440,5050,4713,-7
2,New London County:9,9613,9612,9546,9289,8956,8470,8118,7607,-7
3,Tolland County:9,5423,5423,5373,5220,4999,4725,4475,4185,-7
4,Middlesex County:2,4965,4965,4937,4892,4624,4460,4198,3954,-6
5,New Haven County:9,29678,29678,29492,28845,28184,27140,26035,24843,-5
6,Windham County:9,4227,4227,4227,4246,4170,4057,3861,3670,-5


In [191]:
women_change[women_change["diff_15"] >= 5].sort_values(by="diff_15").reset_index()

YEAR,id,1,2,3,4,5,6,7,8,diff_15
0,Middlesex County:14,4221,4221,4237,4314,4697,4787,4876,5159,5
1,Windham County:14,2469,2469,2486,2602,2832,2885,3036,3213,5


In [192]:
women_change[women_change["diff_15"] <= -5].sort_values(by="diff_15").reset_index()

YEAR,id,1,2,3,4,5,6,7,8,diff_15
0,Litchfield County:9,7392,7392,7327,7105,6787,6388,5897,5381,-9
1,Tolland County:9,5427,5426,5365,5221,5005,4715,4336,4031,-8
2,Middlesex County:9,6497,6497,6442,6281,6056,5773,5424,5049,-7
3,New London County:9,10128,10127,10016,9652,9341,8931,8397,7872,-7
4,New Haven County:9,31464,31463,31336,30963,30223,29383,28096,26564,-6
5,Windham County:9,4240,4241,4231,4178,4128,4015,3803,3639,-5


In [199]:
men_change = men_change.reset_index()

In [208]:
men_change[(men_change["id"].str.contains("Litchfield"))].sort_values(by="diff_15")

YEAR,id,1,2,3,4,5,6,7,8,diff_15
56,Litchfield County:9,7165,7165,7094,6807,6399,6066,5635,5150,-9
39,Litchfield County:1,4791,4791,4768,4628,4421,4219,4061,3928,-4
40,Litchfield County:10,8312,8312,8275,8085,7793,7462,7150,6882,-4
50,Litchfield County:3,6428,6428,6412,6333,6126,5985,5839,5611,-4
51,Litchfield County:4,6416,6416,6425,6431,6334,6202,5991,5855,-3
41,Litchfield County:11,8638,8638,8639,8626,8436,8234,8093,7953,-2
54,Litchfield County:7,4335,4335,4358,4423,4490,4553,4647,4600,-2
49,Litchfield County:2,5684,5684,5639,5477,5272,5111,4864,4787,-2
38,Litchfield County:0,93338,93338,93292,92991,92242,91908,91112,90563,-1
42,Litchfield County:12,7601,7601,7659,7812,8001,8232,8246,8239,-1


In [245]:
totals = census[(census["AGEGRP"] == 0)][["CTYNAME","YEAR","TOT_POP"]]
totals.pivot(index="CTYNAME", columns="YEAR",values="TOT_POP").reset_index()

YEAR,CTYNAME,1,2,3,4,5,6,7,8
0,Fairfield County,916829,916850,919744,929013,935238,941793,945816,948053
1,Hartford County,894014,894029,895303,896928,897149,897926,896871,895841
2,Litchfield County,189927,189927,189773,188945,187455,186609,184910,183603
3,Middlesex County,165676,165676,165628,166198,165508,165301,164757,164063
4,New Haven County,862477,862474,863401,863977,863921,862515,861238,859470
5,New London County,274055,274046,274067,273211,274394,273716,272743,271863
6,Tolland County,152691,152682,153208,153069,151934,151722,151597,151420
7,Windham County,118428,118434,118593,118418,117942,117586,116830,116573
